In [1]:
"""%%bash
wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470
' -O KuaiRec.zip
unzip KuaiRec.zip -d data_final_project"""

"%%bash\nwget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470\n' -O KuaiRec.zip\nunzip KuaiRec.zip -d data_final_project"

# Imports

In [1]:
SEED = 69
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from typing import Tuple, Dict
import plotly.express as px
import plotly.figure_factory as ff
from scipy import stats
import ast
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Lambda
import tensorflow.keras.backend as K
#######################################
np.random.seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-05-13 17:59:34.896236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747151974.906219   60248 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747151974.909045   60248 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747151974.917627   60248 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747151974.917639   60248 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747151974.917640   60248 computation_placer.cc:177] computation placer alr

# Loading the data

In [2]:
root = "data_final_project/KuaiRec 2.0/data/"
small_matrix = pd.read_csv(root + "small_matrix.csv").drop_duplicates().dropna()
social_network = pd.read_csv(root + "social_network.csv").drop_duplicates().dropna()
user_features = pd.read_csv(root + "user_features.csv").drop_duplicates().dropna()
item_daily_features = pd.read_csv(root + "item_daily_features.csv").drop_duplicates().dropna()
item_categories = pd.read_csv(root + "item_categories.csv").drop_duplicates().dropna()
caption_category = pd.read_csv(root + "kuairec_caption_category.csv", lineterminator='\n').drop_duplicates().dropna()
# There is a category id equal to -124 and no category being 30, so I set the negative one to be 30 for practical reasons
caption_category['first_level_category_id'] = [x if x >= 0 else 30 for x in caption_category['first_level_category_id']]

# Data Cleaning

##### Cleaning steps:
- small/big_matrix: watch_ratio (we remove unrealistic ones)
- social_network: change friend list to actual list type (not str), add a column for friends count
- item_categories: we explode feats and encode them
- item_daily_features: we keep only NORMAL videos because we do not want to recommend ads, we keep only videos which are in public visible status because we can't recommend a private video, we also apply cumulative sums on the data to get the stats up to a certain day
- user_features: good as-is
- caption_category: good as-is

In [4]:
known_user_ids = set(user_features["user_id"].unique())
known_video_ids = set(caption_category["video_id"].unique()) & set(item_categories["video_id"].unique()) & set(item_daily_features["video_id"].unique())
print(f"We have data for {len(known_user_ids)} users and {len(known_video_ids)} videos")

We have data for 6899 users and 8846 videos


#### small_matrix:

In [5]:
small_matrix_cleaned = small_matrix.copy()
small_matrix_cleaned = small_matrix_cleaned.drop(columns=["play_duration", "date", "time"])
# We remove duplicates, null values and filter out unrealistic watch_ratio values
small_matrix_cleaned = small_matrix_cleaned.dropna().drop_duplicates()
small_matrix_cleaned = small_matrix_cleaned[small_matrix_cleaned.loc[:, "watch_ratio"] < 5]
# We remove interactions with unknown users or unknown videos
small_matrix_cleaned = small_matrix_cleaned[small_matrix_cleaned.loc[:, "user_id"].isin(known_user_ids)]
small_matrix_cleaned = small_matrix_cleaned[small_matrix_cleaned.loc[:, "video_id"].isin(known_video_ids)]
# We convert times to actual times
small_matrix_cleaned["timestamp"] = pd.to_datetime(small_matrix_cleaned["timestamp"], unit="s", errors="coerce")
assert(not small_matrix_cleaned.isnull().sum().any())
small_matrix_cleaned

,user_id,video_id,video_duration,timestamp,watch_ratio
0,14,148,6067,2020-07-04 21:27:48.378000021,0.722103
1,14,183,6100,2020-07-04 21:28:00.056999922,1.907377
2,14,3649,10867,2020-07-04 21:29:09.479000092,2.063311
3,14,5262,7908,2020-07-04 21:30:43.285000086,0.566388
4,14,8234,11000,2020-07-04 21:35:43.459000111,0.418364
...,...,...,...,...,...
4676370,7162,9177,37205,2020-09-01 12:06:35.983999968,0.142857
4676371,7162,4987,8167,2020-09-02 06:44:51.342000008,1.234848
4676372,7162,7988,49319,2020-09-03 00:45:01.473999977,1.024412
4676373,7162,6533,8000,2020-09-04 14:56:32.020999908,0.273750


In [6]:
earliest_date = small_matrix_cleaned["timestamp"].min()
latest_date = small_matrix_cleaned["timestamp"].max()
print(f"Our date ranges from {earliest_date} to {latest_date}")
#train_test_dates = ["2020-07-03", "2020-08-15"]
train_test_dates = ["2020-07-03", "2020-08-20"]
print(f"We will split our training and testing sets on the {train_test_dates[1]} to keep data temporality")
print(len(small_matrix_cleaned[(small_matrix_cleaned["timestamp"] < train_test_dates[1])]))
print(len(small_matrix_cleaned[(small_matrix_cleaned["timestamp"] >= train_test_dates[1])]))

Our date ranges from 2020-07-03 18:23:26.059999943 to 2020-09-05 15:57:23.683000088
We will split our training and testing sets on the 2020-08-20 to keep data temporality
3414860
455362


#### social_network:

In [7]:
social_network_cleaned = social_network.copy()
# We remove unknown users
social_network_cleaned = social_network_cleaned[social_network_cleaned.loc[:, "user_id"].isin(known_user_ids)]
# Convert string to actual list of ints
social_network_cleaned["friend_list"] = social_network_cleaned["friend_list"].apply(ast.literal_eval)
# Add friend_count row
social_network_cleaned["friend_count"] = social_network_cleaned["friend_list"].apply(lambda x : len(x))
# Add users with no friends for consistency
missing_ids = [id for id in known_user_ids if id not in social_network_cleaned["user_id"].unique()]
missing_users_social = pd.DataFrame({"user_id": missing_ids, "friend_list": [[] for _ in range(len(missing_ids))], "friend_count": [0] * len(missing_ids)})
social_network_cleaned = pd.concat([social_network_cleaned, missing_users_social], ignore_index=True)
assert(len(social_network_cleaned) == len(known_user_ids))
social_network_cleaned

,user_id,friend_list,friend_count
0,3371,[2975],1
1,24,[2665],1
2,4402,[38],1
3,4295,[4694],1
4,7087,[7117],1
...,...,...,...
6894,7170,[],0
6895,7171,[],0
6896,7172,[],0
6897,7173,[],0


#### item_categories:

In [8]:
item_categories_cleaned = item_categories.copy()
item_categories_cleaned = item_categories_cleaned[item_categories_cleaned.loc[:, "video_id"].isin(known_video_ids)]
item_categories_cleaned["feat"] = item_categories_cleaned["feat"].apply(ast.literal_eval)
item_categories_cleaned = item_categories_cleaned.explode("feat")
# Create the vector encoding
item_categories_cleaned = item_categories_cleaned.assign(present=1).pivot(index="video_id", columns="feat", values="present").fillna(0).reindex(columns=sorted(item_categories_cleaned["feat"].unique())).astype(int)
item_categories_cleaned.columns = [f"feat_{feat}" for feat in item_categories_cleaned.columns]
item_categories_cleaned = item_categories_cleaned.reset_index()
assert(len(item_categories_cleaned["video_id"]) == len(known_video_ids))
item_categories_cleaned

,video_id,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8841,10722,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8842,10723,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8843,10724,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8844,10726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### item_daily_features:

In [9]:
item_daily_features_cleaned = item_daily_features.copy()
item_daily_features_cleaned = item_daily_features_cleaned[item_daily_features_cleaned["visible_status"] == "public"]
item_daily_features_cleaned = item_daily_features_cleaned[item_daily_features_cleaned["video_type"] == "NORMAL"]
item_daily_features_cleaned["date"] = pd.to_datetime(item_daily_features_cleaned["date"].astype(str), errors="coerce")
stats_cols = ["show_cnt", "valid_play_cnt", "like_cnt", "comment_cnt", "share_cnt", "follow_cnt", "collect_cnt", "download_cnt", "cancel_like_cnt", "delete_comment_cnt", "cancel_follow_cnt", "cancel_collect_cnt"]
useful_cols = ["video_id", "date"] + stats_cols
item_daily_features_cleaned = item_daily_features_cleaned[useful_cols]
item_daily_features_cleaned = item_daily_features_cleaned.sort_values(["date", "video_id"])
item_daily_features_cleaned[stats_cols] = item_daily_features_cleaned.groupby("video_id")[stats_cols].cumsum()
item_daily_features_cleaned

,video_id,date,show_cnt,valid_play_cnt,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,download_cnt,cancel_like_cnt,delete_comment_cnt,cancel_follow_cnt,cancel_collect_cnt
23,0,2020-07-28,13957,4280,462,10,0,247,2.0,3,85,0,0,0.0
149,2,2020-07-28,95,7,1,0,0,0,0.0,0,4,0,0,0.0
274,4,2020-07-28,9,0,0,0,0,0,0.0,0,0,0,0,0.0
327,5,2020-07-28,1016,190,37,2,2,4,5.0,0,12,0,0,2.0
390,6,2020-07-28,281,6,1,0,0,0,0.0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343336,10723,2020-09-05,277,114,24,0,0,0,0.0,0,0,0,0,0.0
343337,10724,2020-09-05,1100,754,264,29,1,8,0.0,1,14,0,0,0.0
343338,10725,2020-09-05,16996,9317,851,36,3,12,0.0,5,19,0,0,0.0
343339,10726,2020-09-05,7644,5382,44,0,1,2,0.0,2,1,0,0,0.0


#### user_features:

In [10]:
user_features_cleaned = user_features.copy()
useful_cols = ["user_id", "user_active_degree", "is_lowactive_period", "is_live_streamer", "is_video_author", "follow_user_num", "fans_user_num", "friend_user_num", "register_days"]
useful_cols += [f"onehot_feat{i}" for i in range(18)]
user_features_cleaned = user_features_cleaned[useful_cols]
#user_features_cleaned

#### caption_category

In [11]:
caption_category_cleaned = caption_category.copy()
useful_cols = ["video_id", "first_level_category_id"]
caption_category_cleaned = caption_category_cleaned[useful_cols]
caption_category_cleaned

,video_id,first_level_category_id
0,0,8
2,2,9
3,3,26
4,4,5
5,5,6
...,...,...
10722,10722,5
10723,10723,33
10724,10724,6
10726,10726,38


# Aggregating Train and Test Data

In [12]:
train_data = small_matrix_cleaned.copy()[small_matrix_cleaned["timestamp"] < train_test_dates[1]].sort_values(["timestamp", "video_id"])
test_data = small_matrix_cleaned.copy()[small_matrix_cleaned["timestamp"] >= train_test_dates[1]].sort_values(["timestamp", "video_id"])

train_data = train_data.merge(social_network_cleaned, on="user_id", how="left")
train_data = train_data.merge(item_categories_cleaned, on="video_id", how="left")
train_data = train_data.merge(user_features_cleaned, on="user_id", how="left")
train_data = train_data.merge(caption_category_cleaned, on="video_id", how="left")
train_data = pd.merge_asof(train_data, item_daily_features_cleaned, by="video_id", left_on="timestamp", right_on="date", direction="backward")
train_data = train_data.fillna(0)

test_data = test_data.merge(social_network_cleaned, on="user_id", how="left")
test_data = test_data.merge(item_categories_cleaned, on="video_id", how="left")
test_data = test_data.merge(user_features_cleaned, on="user_id", how="left")
test_data = test_data.merge(caption_category_cleaned, on="video_id", how="left")
test_data = pd.merge_asof(test_data, item_daily_features_cleaned, by="video_id", left_on="timestamp", right_on="date", direction="backward")
test_data = test_data.fillna(0)

In [13]:
train_data[train_data["video_id"] == 9912]

,user_id,video_id,video_duration,timestamp,watch_ratio,friend_list,friend_count,feat_0,feat_1,feat_2,...,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,download_cnt,cancel_like_cnt,delete_comment_cnt,cancel_follow_cnt,cancel_collect_cnt
1393680,5888,9912,10700,2020-07-25 00:43:30.329999924,0.540467,[],0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1411824,3039,9912,10700,2020-07-25 05:25:38.641000032,1.550093,[],0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1422964,5361,9912,10700,2020-07-25 09:24:53.565999985,0.707944,[],0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438457,4461,9912,10700,2020-07-25 17:58:20.155999899,0.680748,[],0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1484505,1132,9912,10700,2020-07-26 08:03:25.819000006,1.233738,[],0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396465,6789,9912,10700,2020-08-19 17:00:30.710999966,0.194486,[],0,0,0,0,...,15334.0,71.0,539.0,4173.0,298.0,0.0,1112.0,1.0,0.0,8.0
3405573,3144,9912,10700,2020-08-19 20:24:35.773000002,0.465327,[],0,0,0,0,...,15334.0,71.0,539.0,4173.0,298.0,0.0,1112.0,1.0,0.0,8.0
3406448,5860,9912,10700,2020-08-19 20:48:20.285000086,1.135794,[],0,0,0,0,...,15334.0,71.0,539.0,4173.0,298.0,0.0,1112.0,1.0,0.0,8.0
3407221,724,9912,10700,2020-08-19 21:07:00.013999939,1.073645,[],0,0,0,0,...,15334.0,71.0,539.0,4173.0,298.0,0.0,1112.0,1.0,0.0,8.0


In [14]:
train_data = train_data.drop(columns=["date"])
test_data = test_data.drop(columns=["date"])
export_dir = "./exports"
if not os.path.exists(export_dir):
    os.makedirs(export_dir)
train_data.to_parquet(export_dir + "/train_data.pq")
test_data.to_parquet(export_dir + "/test_data.pq")

In [15]:
train_data.columns

Index(['user_id', 'video_id', 'video_duration', 'timestamp', 'watch_ratio',
       'friend_list', 'friend_count', 'feat_0', 'feat_1', 'feat_2', 'feat_3',
       'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10',
       'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16',
       'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22',
       'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28',
       'feat_29', 'feat_30', 'user_active_degree', 'is_lowactive_period',
       'is_live_streamer', 'is_video_author', 'follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days', 'onehot_feat0',
       'onehot_feat1', 'onehot_feat2', 'onehot_feat3', 'onehot_feat4',
       'onehot_feat5', 'onehot_feat6', 'onehot_feat7', 'onehot_feat8',
       'onehot_feat9', 'onehot_feat10', 'onehot_feat11', 'onehot_feat12',
       'onehot_feat13', 'onehot_feat14', 'onehot_feat15', 'onehot_feat16',
       'onehot_feat17', 'first_level_catego